# Deep-Learning et génomique: classification de séquences par réseaux de convolution



Dans ce notebook nous allons voir comment **mettre en oeuvre des architectures de type CNN** pour la classification de données génomiques.

Nous travaillerons pour cela sur un jeu de données tiré du papier introduisant la méthode **"DeepBind"**: [Predicting the sequence specificities of DNA- and RNA-binding proteins by deep learning](https://www.nature.com/articles/nbt.3300).


On commence par charger les packages et par bloquer la graine du générateur aléatoire par souci de reproductibilité. 

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# set random seed
np.random.seed(123)

# 1) Lecture du jeu de données

Le jeu de données se présente sous la forme d'un fichier texte nommé **TF_44_CHIP_100_full_genomic.seq.txt**, contenant 996 séquences génomiques (1ère colonne) et leur catégorie associée (0 ou 1, 2ème colonne).

In [ ]:
# read input file
input_file = 'TF_44_CHIP_100_full_genomic.seq.txt'
meta = pd.read_csv(input_file, sep = '\t', header = None)
print(meta.shape)
meta.head()

# ==> EXERCICE: extraire les séquences du jeu de données dans une liste et leurs labels dans un vecteur numpy. Compter le nombre d'instances disponibles pour chacune de catégories et vérifier que toutes les séquences ont la même longueur.

In [ ]:
# split dataset into labels and seqs


In [ ]:
# check sequences have the same length


# 2) Mise en forme du jeu de données pour keras / tensorflow

Dans cette étude nous allons adopter une représentation de type "one-hot-encoding" consistant à représenter une séquence génomique de longueur $L$ par une matrice de dimension $[L\times 4]$, chaque colonne codant pour la présence d'une base A,T, G ou C à une position donnée. 


Cette opération peut être réalisée par la fonction ci-dessous.

In [ ]:
# define one-hot-encoding function
def DNA_to_onehot(seq):
    options_onehot = {
        'A': [1,0,0,0],
        'C': [0,1,0,0], 
        'G': [0,0,1,0],
        'T': [0,0,0,1]
      }
    # cast to one-hot
    onehot_data = map(lambda e: options_onehot[e], seq.upper())
    return np.array(list(onehot_data))


Pour construire la matrice, il suffit d'appliquer cette fonction à toutes les séquences et de convertir la liste obtenue en matrice numpy.

In [ ]:
one_hot_data = [DNA_to_onehot(x) for x in seqs]
X = np.array(one_hot_data)

# ==> EXERCICE: vérifier que la matrice possède bien les dimensions attendues puis découper le jeu de données en jeu d'apprentissage (75%) + jeu de test (25%).

In [ ]:
# check matrix dimension


In [ ]:
# split in training / test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(  )

In [ ]:
print('training / test datasets of sizes {} / {}'.format(X_train.shape,X_test.shape))

# 3) Apprentissage de modèles CNN

Nous allons maintenant chercher à mettre en oeuvre deux modèles à bases de filtres de convolution: 
* un modèle linéaire, mettant en jeu une couche de convolution, un "pooling" global et une couche de sortie.
* un modèle non linéaire, mettant en jeu une couche "dense" supplémentaire avec la couche de sortie.

La syntaxe à utiliser sera sensiblement la même que pour les CNNs appliqués à l'imagerie.
A noter néanmoins:
* que la couche de convolution à utiliser sera la couche [Conv1D](https://keras.io/api/layers/convolution_layers/convolution1d/). En effet, bien que l'on ait transformé notre image en une matrice, il nous est possible de faire appel à une couche de convolution 1D. Dans ce cas, il nous suffit de spécifier (1) le nombre de filtres et (2) la longueur de ces filtres (en terme de bases nucléiques). Le second paramètre de taille des masques de convolution (ici 4, pour les 4 bases ATGC) est déterminé automatiquement.
* que le "pooling global" peut être réalisé par la couche [GlobalMaxPooling1D](https://keras.io/api/layers/pooling_layers/global_max_pooling1d/), ce qui aura pour effet de considérer la valeur maximale capturée par chaque filtre le long de la séquence.

# ==> EXERCICE: constuire les deux modèles, et faire appel à la méthode "summary()" pour bien comprendre la dimension obtenues en sortie des différentes couches.


### définition des architectures - avec et sans MLP

In [ ]:
# specify number of filters and filter width
n_filters = 20
filter_width = 7
# sequence length
seq_len = X_train.shape[1]

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D

In [ ]:
# define CNN architecture - linear 
cnn_lin = Sequential()


In [ ]:
cnn_lin.summary()

In [ ]:
# define CNN architecture - non-linear 
cnn_nonlin = Sequential()


In [ ]:
cnn_nonlin.summary()

### définition des paramètres d'apprentissage

In [ ]:
cnn_lin.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 
cnn_nonlin.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

### apprentissage des modèles

In [ ]:
# specify number of epochs
n_epochs = 300

In [ ]:
# fit linear model
history_lin = cnn_lin.fit(X_train, y_train, batch_size=32, epochs=n_epochs, verbose=1, validation_split=0.2)

In [ ]:
plt.figure(figsize=(12,4))
# show accuracy
plt.subplot(1,2,1)
plt.plot(history_lin.history['accuracy'], label = 'train')
plt.plot(history_lin.history['val_accuracy'], label = 'validation')
plt.title("linear CNN - accuracy vs epochs")
plt.grid()
plt.legend()
# show loss
plt.subplot(1,2,2)
plt.plot(history_lin.history['loss'], label = 'train')
plt.plot(history_lin.history['val_loss'], label = 'validation')
plt.title("linear CNN - loss vs epochs")
plt.grid()
plt.legend()
plt.show()

In [ ]:
# fit non-linear model
history_nonlin = cnn_nonlin.fit(X_train, y_train, batch_size=32, epochs=n_epochs, verbose=1, validation_split=0.2)

In [ ]:
plt.figure(figsize=(12,4))
# show accuracy
plt.subplot(1,2,1)
plt.plot(history_nonlin.history['accuracy'], label = 'train')
plt.plot(history_nonlin.history['val_accuracy'], label = 'validation')
plt.title("non-linear CNN - accuracy vs epochs")
plt.grid()
plt.legend()
# show loss
plt.subplot(1,2,2)
plt.plot(history_nonlin.history['loss'], label = 'train')
plt.plot(history_nonlin.history['val_loss'], label = 'validation')
plt.title("non-linear CNN - loss vs epochs")
plt.grid()
plt.legend()
plt.show()

# 4) Pour aller plus loin....


On peut imaginer de nombreuses choses pour aller plus loin : 
* évaluer l'impact des paramètres (nombre de filtres, longueur des filtres, structure de MLP, ...)...et pourquoi pas, le faire via TensorBoard.
* investiguer la structure des masques appris (par exemple en utilisant le package [LogoMaker](https://logomaker.readthedocs.io/en/latest/))
* considérer des architectures plus complexes, mettant en jeu des couches récurrentes (avant ou à la place du "pooling" global), et/ou une représentation sous forme de k-mers.

Enfin, bien entendu, enregistrer le meilleur modèle obtenu sur le jeu de validation (e.g., via un callBack de ModelCheckpoint) et  mesurer les performances de prédiction sur le jeu de test.
